In [2]:
import os
import sys

sys.path.append("../jupyter-py/")
from decode_logs import *
sys.path.pop()

output_dir = "../jupyter-py/output/" + get_current_time()
strategy_type = "kalman"
execution_command = "python ../jupyter-py/backtest_pair.py --strategy_type {} --output_dir {}".format(strategy_type, output_dir)

os.system("rm -rf ../jupyter-py/output")
os.system(execution_command)


0

In [5]:
df = Decoder.get_strategy_status(output_dir)

In [7]:
# normalized price, figure name = normp_p

import pandas as pd
import numpy as np

from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool, DatetimeTickFormatter
from bokeh.plotting import figure

# ========== themes & appearance ============= #

STK_1_LINE_COLOR = "#053061"
STK_2_LINE_COLOR = "#67001f"
STK_1_LINE_WIDTH = 1.5
STK_2_LINE_WIDTH = 1.5
WINDOW_SIZE = 10000
TITLE = "NORM PRICE OF X vs Y" 


# ========== data ============= #
# use sample data from ib-data folder
STK_1 = pd.DataFrame.from_csv("../ib-data/nyse/AAN.csv").reset_index()
STK_2 = pd.DataFrame.from_csv("../ib-data/nyse/AER.csv").reset_index()

STK_1_dates = np.array(STK_1['date'], dtype=np.datetime64)
STK_1_source = ColumnDataSource(data=dict(date=STK_1_dates, close=STK_1['close']))

STK_2_dates = np.array(STK_2['date'], dtype=np.datetime64)
STK_2_source = ColumnDataSource(data=dict(date=STK_2_dates, close=STK_2['close']))

# ========== plot data points ============= #
# x_range is the zoom in slider setup. Pls ensure both STK_1 and STK_2 have same length, else some issue
normp = figure(plot_height=250, plot_width=600, x_range=(STK_1_dates[-WINDOW_SIZE], STK_1_dates[-1]), title=TITLE, toolbar_location=None)

normp.line('date', 'close', source=STK_1_source, line_color = STK_1_LINE_COLOR, line_width = STK_1_LINE_WIDTH)
normp.line('date', 'close', source=STK_2_source, line_color = STK_2_LINE_COLOR, line_width = STK_2_LINE_WIDTH)
normp.yaxis.axis_label = 'Price'

normp.xaxis[0].formatter = DatetimeTickFormatter()


# ========== RANGE SELECT TOOL ============= #

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=100, plot_width=600, y_range=normp.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=normp.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.line('date', 'close', source=STK_1_source, line_color = STK_1_LINE_COLOR, line_width = STK_1_LINE_WIDTH)
select.line('date', 'close', source=STK_2_source, line_color = STK_2_LINE_COLOR, line_width = STK_2_LINE_WIDTH)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

normp_p = column(normp, select)

/Users/brendantham/anaconda3/envs/fyp/lib/python3.5/site-packages/ipykernel/__main__.py:23: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
/Users/brendantham/anaconda3/envs/fyp/lib/python3.5/site-packages/ipykernel/__main__.py:24: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [14]:
# spread plot, figure name = spread_p

import pandas as pd

from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models.glyphs import Line
from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.periodic_table import elements
import numpy as np

# ========== themes & appearance ============= #

palette = ["#053061", "#2166ac", "#4393c3", "#92c5de", "#d1e5f0",
           "#f7f7f7", "#fddbc7", "#f4a582", "#d6604d", "#b2182b", "#67001f"]

LINE_WIDTH = 1.5
LINE_COLOR = palette[-1]
TITLE = "RULE BASED SPREAD TRADING"

# ========== data ============= #

elements = elements.copy()
elements = elements[elements["atomic number"] <= 30]
elements = elements[~pd.isnull(elements["melting point"])]
mass = [float(x.strip("[]")) for x in elements["atomic mass"]]
elements["atomic mass"] = mass

melting_points = elements["melting point"]
low = min(melting_points)
high = max(melting_points)
melting_point_inds = [int(10*(x-low)/(high-low)) for x in melting_points] #gives items in colors a value from 0-10
elements['melting_colors'] = [palette[i] for i in melting_point_inds]

# ========== figure INTERACTION properties ============= #

TOOLS = "hover,pan,wheel_zoom,box_zoom,reset,save"

spread_p = figure(tools=TOOLS, toolbar_location="above", plot_height=250, plot_width=600, title=TITLE)
spread_p.toolbar.logo = "grey"
spread_p.background_fill_color = "#dddddd"
spread_p.xaxis.axis_label = "atomic weight (amu)"
spread_p.yaxis.axis_label = "density (g/cm^3)"
spread_p.grid.grid_line_color = "white"

# you need to @ the column name of the dataframe, and you pass the dataframe as a "source" variable (see below)
# format: <name that appears>, @<column_name>
spread_p.hover.tooltips = [
    ("name", "@name"),
    ("symbol:", "@symbol"),
    ("density", "@density"),
    ("atomic weight", "@{atomic mass}"),
    ("melting point", "@{melting point}")
]


# ========== plot data points ============= #
# this part, atomic_mass = x-axis (vice versa from y)
# you just put in column name

source = ColumnDataSource(elements)

# plot the POINT coords of the ACTIONS
spread_p.circle("atomic mass", "density", size=12, source=source,
         color='melting_colors', line_color="black", fill_alpha=0.8)

# plot the spread over time
spread_p.line(x = np.array(elements["atomic mass"]), y = np.array(elements["density"]), line_color = LINE_COLOR, line_width = LINE_WIDTH)

# ========== plot label ============= #
# this part you just need to pass BUY or SELL actions
# recommend you use one colour for each action
# x = <col_name_x-axis>, y = <col_name_y-axis> // both from the source dataframe

labels = LabelSet(x="atomic mass", y="density", text="symbol", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')

spread_p.add_layout(labels)

# ========== RANGE SELECT TOOL ============= #
# not included for now because sample data x-axis is not datetime. PLS FIX

# select = figure(title="Drag the middle and edges of the selection box to change the range above",
#                 plot_height=130, plot_width=600, y_range=spread_p.y_range, background_fill_color="#efefef")

# range_tool = RangeTool(x_range=spread_p.x_range)
# range_tool.overlay.fill_color = "navy"
# range_tool.overlay.fill_alpha = 0.2

# select.line('atomic mass', 'density', source=source, line_color = LINE_COLOR, line_width = LINE_WIDTH)
# select.ygrid.grid_line_color = None
# select.add_tools(range_tool)
# select.toolbar.active_multi = range_tool

# show(column(spread_p,select))

In [15]:
# PV line chart pv_p

# normalized price, figure name = normp

import pandas as pd
import numpy as np

from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool, DatetimeTickFormatter
from bokeh.plotting import figure

# ========== themes & appearance ============= #

LINE_COLOR = "#053061"
LINE_WIDTH = 1.5
TITLE = "PORTFOLIO VALUE OVER TIME" 


# ========== data ============= #
# use sample data from ib-data folder
STK_1 = pd.DataFrame.from_csv("../ib-data/nyse/AAN.csv").reset_index()

STK_1_dates = np.array(STK_1['date'], dtype=np.datetime64)
STK_1_source = ColumnDataSource(data=dict(date=STK_1_dates, close=STK_1['close']))

# ========== plot data points ============= #
# x_range is the zoom in slider setup. Pls ensure both STK_1 and STK_2 have same length, else some issue
pv_p = figure(plot_height=250, plot_width=600, title=TITLE, toolbar_location=None)

pv_p.line('date', 'close', source=STK_1_source, line_color = LINE_COLOR, line_width = LINE_WIDTH)
pv_p.yaxis.axis_label = 'Price'

pv_p.xaxis[0].formatter = DatetimeTickFormatter()



/Users/brendantham/anaconda3/envs/fyp/lib/python3.5/site-packages/ipykernel/__main__.py:22: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [16]:
from datetime import date

# widgets
from bokeh.layouts import column
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Button, Select, DateRangeSlider

WIDTH = 250

# ========== Select Stocks ============= #
stock_list = ['AAPL', 'GOOGL', 'IBM']
select_stk_1 = Select(width = WIDTH, title='Select Stock 1:', value = stock_list[0], options=stock_list)
select_stk_2 = Select(width = WIDTH, title='Select Stock 2:', value = stock_list[0], options=stock_list)


# ========== Strategy Type ============= #
strategy_list = ['Make money', 'Do Cocaine', 'Models and Bottles']
select_strategy = Select(width = WIDTH, title='Select Strategy:', value = strategy_list[0], options=strategy_list)

# ========== set start/end date ============= #
# date time variables
MAX_START = date(2014, 1, 1)
MAX_END = date(2018, 12, 30)
DEFAULT_START =date(2016, 1, 1)
DEFAULT_END = date(2018, 1, 30)
STEP = 1

backtest_dates = DateRangeSlider(width = WIDTH, start=MAX_START, end=MAX_END, value=(DEFAULT_START, DEFAULT_END), step=STEP, title="Backtest Date Range:")

start_bt = Button(label="Backtest", button_type="success", width = WIDTH)

# controls = column(select_stk_1, select_stk_2, select_strategy, backtest_dates, start_bt)
controls_wb = widgetbox(select_stk_1, select_stk_2, select_strategy, backtest_dates, start_bt, width=600)


In [17]:
from bokeh.layouts import gridplot
grid = gridplot([[controls_wb, normp_p], [pv_p, spread_p]], sizing_mode='fixed')
# mid = column(normp_p, spread_p, pv_p)

In [18]:
show(grid)

In [20]:
show(spread_p)